In [1]:
%load_ext autoreload
%autoreload

In [2]:
import argparse
import os
import sys
import torch
import torchvision
import numpy as np
from torch.backends import cudnn
from torch import optim
from torch.autograd import Variable
from torch.utils import data
from torchvision import transforms
from torchvision import datasets
from PIL import Image
from InfoGan import *

import matplotlib.pyplot as plt

In [3]:
##### Helper Function for Image Loading
class ImageFolder(data.Dataset):
    def __init__(self, root, transform=None):  # Initializes image paths and preprocessing module.
        self.image_paths = list(map(lambda x: os.path.join(root, x), os.listdir(root)))
        self.transform = transform

    def __getitem__(self, index):  # Reads an image from a file and preprocesses it and returns.
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image

    def __len__(self):  # Returns the total number of image files.
        return len(self.image_paths)

##### Helper Function for GPU Training
def to_variable(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

##### Helper Function for Math
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

# InfoGAN Function (Gaussian)
def gen_cc(n_size, dim):
    return torch.Tensor(np.random.randn(n_size, dim) * 0.5 + 0.0)

# InfoGAN Function (Multi-Nomial)
def gen_dc(n_size, dim):
    codes=[]
    code = np.zeros((n_size, dim))
    random_cate = np.random.randint(0, dim, n_size)
    code[range(n_size), random_cate] = 1
    codes.append(code)
    codes = np.concatenate(codes,1)
    return torch.Tensor(codes)

In [4]:
# cudnn.benchmark = True

params = {}
params['image_size'] = 28 # 28 for mnist
params['z_dim'] = 62
params['batch_size'] = 64
params['num_workers'] = 4
params['num_epochs'] = 30

#optimization params
params['lrD'] = 0.0002  # Learning rate for Discriminator
params['lrG'] = 0.001   # Learning rate for Generator
params['beta1'] = 0.5   # Momentum 1 in Adam
params['beta2'] = 0.999 # Momentum 2 in Adam

#infoGan params
params['cc_dim'] = 1
params['dc_dim'] = 10
params['continuous_weight'] = 0.5

#misc
params['db'] = 'mnist'
params['sample_size'] = 100
params['model_path'] = './model'
params['sample_path'] = './results'
params['log_step'] = 50
params['sample_step'] = 100

In [5]:
#create dataset
transform = transforms.Compose([
    transforms.Resize((params['image_size'], params['image_size'])),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
dataset = datasets.MNIST('./MNIST', train=True, transform=transform, target_transform=None, download=True)

In [6]:
data_loader = data.DataLoader(dataset = dataset,
                              batch_size = params['batch_size'],
                              shuffle = True,
                              num_workers = params['num_workers'])

In [7]:
if not os.path.exists(params['model_path']):
    os.makedirs(params['model_path'])
if not os.path.exists(params['sample_path']):
    os.makedirs(params['sample_path'])

In [8]:
generator = Generator(params['db'], params['z_dim'], params['cc_dim'], params['dc_dim'])
discriminator = Discriminator(params['db'], params['cc_dim'], params['dc_dim'])

In [9]:
g_optimizer = optim.Adam(generator.parameters(), params['lrD'], [params['beta1'], params['beta2']])
d_optimizer = optim.Adam(discriminator.parameters(), params['lrG'], [params['beta1'], params['beta2']])

In [10]:
if torch.cuda.is_available():
    generator.cuda()
    discriminator.cuda()

In [11]:
fixed_noise = to_variable(torch.Tensor(np.zeros((params['sample_size'], params['z_dim']))))

In [12]:
total_step = len(data_loader) 
for epoch in range(params['num_epochs']):
    print('')
    for i,  images in enumerate(data_loader):
        # ===================== Train Discriminator =====================#
        images = to_variable(images[0]) # remove Label

        batch_sz = images.size(0)
        noise = to_variable(gen_cc(batch_sz, params['z_dim']))
        
        cc = to_variable(gen_cc(batch_sz, params['cc_dim']))
        dc = to_variable(gen_cc(batch_sz, params['dc_dim']))
        
        # Fake -> Fake & Real -> Real
        fake_images = generator(torch.cat((noise, cc, dc),1))
        d_output_real = discriminator(images)
        d_output_fake = discriminator(fake_images)
        
        d_loss_a = -torch.mean(torch.log(d_output_real[:,0]) + torch.log(1 - d_output_fake[:,0]))
        
        # Mutual Information Loss
        output_cc = d_output_fake[:, 1:1 + params['cc_dim']]
        output_dc = d_output_fake[:,   1 + params['cc_dim']:]
        d_loss_cc = torch.mean((((output_cc - 0.0) / 0.5) ** 2))
        d_loss_dc = -(torch.mean(torch.sum(dc * output_dc, 1)) + torch.mean(torch.sum(dc * dc, 1)))
        
        d_loss = d_loss_a + params['continuous_weight'] * d_loss_cc + 1.0 * d_loss_dc
        
        # Optimization
        discriminator.zero_grad()
        d_loss.backward(retain_graph=True)
        d_optimizer.step()
        
        # ===================== Train Generator =====================#
        g_loss_a = -torch.mean(torch.log(d_output_fake[:,0]))

        g_loss = g_loss_a + params['continuous_weight'] * d_loss_cc + 1.0 * d_loss_dc

        # Optimization
        generator.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        # print the log info
        if (i + 1) % params['log_step'] == 0 or (i + 1) == total_step:
            sys.stdout.write('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, g_loss: %.4f\r'
                  % (epoch + 1, params['num_epochs'], i + 1, total_step, d_loss.item(), g_loss.item()))
            sys.stdout.flush()

        # save the sampled images (10 Category(Discrete), 10 Continuous Code Generation : 10x10 Image Grid)
        if (i + 1) % params['sample_step'] == 0:
            tmp = np.zeros((params['sample_size'], params['cc_dim']))
            for k in range(10):
                tmp[k * 10:(k + 1) * 10, 0] = np.linspace(-2, 2, 10)
            cc = to_variable(torch.Tensor(tmp))
            tmp = np.zeros((params['sample_size'], params['dc_dim']))
            for k in range(10):
                tmp[k * 10:(k + 1) * 10, k] = 1
            dc = to_variable(torch.Tensor(tmp))

            fake_images = generator(torch.cat((fixed_noise, cc, dc), 1))
            torchvision.utils.save_image(denorm(fake_images.data),
                                         os.path.join(params['sample_path'],
                                                      'generated-%d-%d.png' % (epoch + 1, i + 1)), nrow=10)

    # save the model parameters for each epoch
    g_path = os.path.join(params['model_path'], 'generator-%d.pkl' % (epoch + 1))
    torch.save(generator.state_dict(), g_path)

/home/jperalta/infogan/InfoGan.py:133: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out[:, self.cc_dim + 1:self.cc_dim + 1 + self.dc_dim] = F.softmax(out[:, self.cc_dim + 1:self.cc_dim + 1 + self.dc_dim].clone())


Epoch [1/30], Step[938/938], d_loss: -2.9793, g_loss: -0.8175
Epoch [2/30], Step[938/938], d_loss: -2.3423, g_loss: 0.15246
Epoch [3/30], Step[938/938], d_loss: -2.3509, g_loss: 0.28777
Epoch [4/30], Step[938/938], d_loss: -2.7280, g_loss: -0.4912
Epoch [5/30], Step[938/938], d_loss: -2.0280, g_loss: -1.0066
Epoch [6/30], Step[938/938], d_loss: -2.1530, g_loss: 0.98776
Epoch [7/30], Step[938/938], d_loss: -2.8184, g_loss: -0.2426
Epoch [8/30], Step[938/938], d_loss: -2.5938, g_loss: -0.5026
Epoch [9/30], Step[938/938], d_loss: -2.7501, g_loss: 1.93634
Epoch [10/30], Step[938/938], d_loss: -2.8228, g_loss: 0.90130
Epoch [11/30], Step[938/938], d_loss: -2.9120, g_loss: -0.3029
Epoch [12/30], Step[938/938], d_loss: -2.6339, g_loss: -1.0085
Epoch [13/30], Step[938/938], d_loss: -3.0885, g_loss: 0.79339
Epoch [14/30], Step[938/938], d_loss: -2.9795, g_loss: 2.11532
Epoch [15/30], Step[938/938], d_loss: -2.7708, g_loss: 0.09476
Epoch [16/30], Step[938/938], d_loss: -2.8009, g_loss: 1.18719
E

In [13]:
images = next(iter(data_loader))[0].cuda()
batch_size = images.size(0)
noise = to_variable(torch.randn(batch_size, params['z_dim']))
images.size()

torch.Size([64, 1, 28, 28])

In [14]:
cc = to_variable(gen_cc(batch_size, params['cc_dim']))
dc = to_variable(gen_dc(batch_size, params['dc_dim']))

In [15]:
# torch.cat((noise, cc, dc),1).size()
fake_images = generator(torch.cat((noise, cc, dc),1))

In [16]:
d_output_real = discriminator(images)

In [1]:
d_output_real

NameError: name 'd_output_real' is not defined

In [ ]:
te